In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPEN_AI_KEY"] = api_key

import random
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [2]:
### BASELINE OUTER ANALYZER GENERATION

# Step 2: Define the prompt template for generating outer analyzers
def create_prompt_template():    
    return PromptTemplate(
        input_variables=["target_prompt_inner_code"],
        template=f"""

You are an AI assistant that generates code for analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
fine-grained mobile network analytics on end device. It is a cross-platform package for mobile network monitoring and analysis.

Given the following prompt and inner analyzer code, generate the corresponding outer analyzer:

{{target_prompt_inner_code}}

NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
WITH NO ADDITIONAL MODIFICATIONS REQUIRED.

Generated Outer Analyzer:
"""
    )

# Step 3: Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Step 4: Function to generate outer analyzer based on in-context examples
def generate_outer_analyzer(target_txt_file, output_folder="generated_outer_analyzers"):
    # Load in-context examples
    
    # Load the target prompt + inner analyzer code from the target .txt file
    with open(target_txt_file, 'r') as f:
        target_prompt_inner_code = f.read().strip()
    
    # Prepare the prompt template
    prompt_template = create_prompt_template()
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Prepare inputs with in-context examples and the target example
    inputs = {}
    inputs["target_prompt_inner_code"] = target_prompt_inner_code

    # Generate the outer analyzer code
    result = chain.invoke(inputs)
    generated_outer_code = result['text'].strip()
    code = generated_outer_code.replace("```python", "", 1)
    code = code.replace("```", "", 1)

    # Save the generated code to a .py file
    os.makedirs(output_folder, exist_ok=True)
    output_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(target_txt_file))[0]}_generated_ICL.py")
    with open(output_filename, 'w') as f:
        f.write(code)
    
    print(f"Generated outer analyzer saved to {output_filename}")

# # Example usage
# target_txt_file = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset\prompt_lte_phy_analyzer_example_1.txt'
# output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers'

# # Generate the outer analyzer
# generate_outer_analyzer(target_txt_file, output_folder=output_folder)


In [3]:
### LOOP THROUGH OUTER ANALYZERS

def process_all_txt_files(example_folder, output_folder):
    # Get all `.txt` files that start with "prompt_"
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    # Iterate through each `.txt` file and call the generate_inner_analyzer function
    for txt_file in txt_files:
        target_txt_file = os.path.join(example_folder, txt_file)
        print(f"Processing {target_txt_file}...")

        # Call the generate_inner_analyzer function
        try:
            generate_outer_analyzer(target_txt_file, output_folder=output_folder)
        except Exception as e:
            print(f"Error processing {txt_file}: {e}")

# Define the input folder containing .txt and .py files
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset'

# Define the output folder where the generated inner analyzers will be saved
output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_baseline'

# Process all .txt files
process_all_txt_files(example_folder, output_folder)

Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset\prompt_dl_retx_analyzer_test_1.txt...


C:\Users\bhull\AppData\Local\Temp\ipykernel_18224\2874162227.py:37: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


Generated outer analyzer saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_baseline\prompt_dl_retx_analyzer_test_1_generated_ICL.py
Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset\prompt_dl_retx_analyzer_test_2.txt...
Generated outer analyzer saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_baseline\prompt_dl_retx_analyzer_test_2_generated_ICL.py
Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset\prompt_dl_retx_analyzer_test_3.txt...
Generated outer analyzer saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_baseline\prompt_dl_

In [3]:
### INNER ANALYZER GENERATION WITH IN-CONTEXT LEARNING

# Define the prompt template for generating inner analyzers
def create_prompt_template():
    
    return PromptTemplate(
        input_variables=["target_prompt_outer_code"],
        template=f"""

You are an AI assistant that generates code for inner analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

Given the following prompt and outer analyzer code, generate the corresponding inner analyzer:

{{target_prompt_outer_code}}

NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
WITH NO ADDITIONAL MODIFICATIONS REQUIRED.

Generated Inner Analyzer:
"""
    )

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Function to generate inner analyzer based on in-context examples
def generate_inner_analyzer(target_txt_file, output_folder="generated_inner_analyzers"):
    
    # Load the target prompt + outer analyzer code from the target .txt file
    with open(target_txt_file, 'r') as f:
        target_prompt_outer_code = f.read().strip()
    
    # Prepare the prompt template
    prompt_template = create_prompt_template()
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Prepare inputs with in-context examples and the target example
    inputs = {}
    inputs["target_prompt_outer_code"] = target_prompt_outer_code

    # Generate the inner analyzer code
    result = chain.invoke(inputs)
    generated_inner_code = result['text'].strip()
    code = generated_inner_code.replace("```python", "", 1)
    code = code.replace("```", "", 1)

    # Save the generated code to a .py file
    os.makedirs(output_folder, exist_ok=True)
    output_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(target_txt_file))[0]}_baseline.py")
    with open(output_filename, 'w') as f:
        f.write(code)
    
    print(f"Generated inner analyzer saved to {output_filename}")

In [5]:
### LOOP THROUGH INNER ANALYZERS

def process_all_txt_files(example_folder, output_folder):
    # Get all `.txt` files that start with "prompt_"
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    # Iterate through each `.txt` file and call the generate_inner_analyzer function
    for txt_file in txt_files:
        target_txt_file = os.path.join(example_folder, txt_file)
        print(f"Processing {target_txt_file}...")

        # Call the generate_inner_analyzer function
        try:
            generate_inner_analyzer(target_txt_file, output_folder=output_folder)
        except Exception as e:
            print(f"Error processing {txt_file}: {e}")

# Define the input folder containing .txt and .py files
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'

# Define the output folder where the generated inner analyzers will be saved
output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers_baseline'

# Process all .txt files
process_all_txt_files(example_folder, output_folder)

Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset\prompt_kpi_manager_1.txt...
Generated inner analyzer saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers_baseline\prompt_kpi_manager_1_baseline.py
Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset\prompt_kpi_manager_2.txt...
Generated inner analyzer saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers_baseline\prompt_kpi_manager_2_baseline.py
Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset\prompt_kpi_manager_2_1.txt...
Generated inner analyzer saved to C:\Users\bhull\Desktop